In [1]:
!pip install tensorboard

In [2]:
from PIL import Image
from PIL import ImageFile
import torch
import glob
import os
import platform
from __future__ import print_function
import zipfile
from zipfile import ZipFile
import json

import argparse
import random
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.tensorboard import SummaryWriter

from functools import partial
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
import torchvision
from Architectures.Conditional_WGAN_net import Generator, Discriminator, weights_init
from Architectures.Conditional_WGAN_net import gradient_penalty
from ImageDataset import ImageDataset

%matplotlib inline
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [16]:
dataset = ImageDataset('DATA')
dataset_info = {'classes': list(set(dataset.labels)), 'encoder': dataset.encoder, 'decoder': dataset.decoder, 'num_classes': dataset.num_classes, 'requires_label': True}

chair_ikea


In [20]:
# Decide which device we want to run on
ngpu = 1
workers = 2
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
learning_rate = 1e-4
beta1 = 0.0
batch_size = 64
image_size = 64
img_ch = 3
num_classes = dataset.num_classes
gen_embedding = 100
z_dim = 100
num_epochs = 1000
features_g = 64
features_d = 64
critic_iterations = 5
lambda_pen = 10

In [18]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [19]:
# Create the Generator and Discriminator
netG = Generator(z_dim=z_dim, img_ch=img_ch, features_g=features_g,
                 num_classes=num_classes, embed_size=gen_embedding,
                 img_size = image_size).to(device)
netD = Discriminator(img_ch=img_ch, features_d=features_d,
                     num_classes=num_classes, img_size=image_size).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    netG = nn.DataParallel(netG, list(range(ngpu)))


netD.apply(weights_init)
netG.apply(weights_init)

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=learning_rate, betas=(beta1, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr=learning_rate, betas=(beta1, 0.9))

In [1]:
fixed_noise = torch.randn(32, z_dim, 1, 1).to(device)
os.makedirs('model/cWGAN', exist_ok=True)
with open('model/cWGAN/image_data.json', 'w') as f:
    json.dump(dataset_info, f)
# Number of training epochs
writer_real = SummaryWriter(log_dir=f'logs_runs/logs_cWGAN/real')
writer_fake = SummaryWriter(log_dir=f'logs_runs/logs_cWGAN/fake')
writer_lossD = SummaryWriter(log_dir=f'logs_runs/logs_cWGAN/lossD')
writer_lossG = SummaryWriter(log_dir=f'logs_runs/logs_cWGAN/lossG')
writer_penalty = SummaryWriter(log_dir=f'logs_runs/logs_cWGAN/penalty')
iters = 0
suffix = f'lr={learning_rate}_beta={beta1}_batch={batch_size}'  
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    for i, (real, labels) in enumerate(dataloader):
        real = real.to(device)
        cur_batch_size = real.shape[0]
        labels = labels.to(device)
        for _ in range(critic_iterations):
            noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
            fake = netG(noise, labels)
            disc_real = netD(real, labels).reshape(-1)
            disc_fake = netD(fake, labels).reshape(-1)
            gp = gradient_penalty(netD, labels, real, fake, device=device)
            errD_real = torch.mean(disc_real)
            errD_fake = torch.mean(disc_fake)
            loss_disc = (-(errD_real - errD_fake)\
                         + lambda_pen * gp)
            netD.zero_grad()
            loss_disc.backward(retain_graph=True)
            optimizerD.step()

        gen_fake = netD(fake, labels).view(-1)
        loss_gen = -torch.mean(gen_fake)
        netG.zero_grad()
        loss_gen.backward()
        optimizerG.step()

        # Output training stats
        if iters % 25 == 0:
            print(
                f'Epoch [{epoch}/{num_epochs}] Batch {i}/{len(dataloader)} '
                + f'Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}'
                )
            with torch.no_grad():
                fake = netG(noise, labels)
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image('Real', img_grid_real, global_step=iters)
                writer_real.add_scalar('D(x)', errD_real, global_step=iters)
                writer_fake.add_image('Fake', img_grid_fake, global_step=iters)
                writer_fake.add_scalar('D(G(z))', errD_fake, global_step=iters)
                writer_lossD.add_scalar('Loss_Discriminator', loss_disc.item(), global_step=iters)
                writer_lossG.add_scalar('Loss_Generator', loss_gen.item(), global_step=iters)
                writer_penalty.add_scalar('Gradient_Penalty', gp.item(), global_step=iters)
        iters += 1
torch.save(netD, 'model/cWGAN/Discriminator.pt')
torch.save(netG, 'model/cWGAN/Generator.pt')

NameError: name 'torch' is not defined